In [1]:
import json
import ipywidgets as widgets
import os
import time
import numpy as np

In [2]:
os.getcwd()

'C:\\Users\\jfwal\\OneDrive\\Documents\\GitHub\\cyberAR\\_Code Device\\AR Labs\\Assets\\Resources\\jsonDefinitions'

In [3]:



# set up the helper routines to load the jsons
# written by J Wallin

def jsonFromFile(fname):
    f = open(fname,"r")
    s = f.read()
    f.close()
    return json.loads(s)

def jsonToFile(fname):
    print(fname)
    
def listFiles(assetPath, printFiles):
    finalList = []
    fullList = os.listdir(assetPath)
    for f in fullList:
        if f.find("json") == len(f)-4:
            print(f)
            finalList.append(f)
    return finalList

def loadFileIntoDictionary(path, flist):
    jdata = {}
    for f in flist:
        jdata[f] = jsonFromFile(path + f)
    return jdata



def writeScene(myScene, verbose):
    # save it to an output file
    outputFile = myScene["jsonFileName"]

    print(os.path.isfile(outputPath + outputFile))

    # if there is a file already there
    print(outputPath + outputFile)
    fileExists = os.path.isfile(outputPath + outputFile)

    if verbose:
        while (fileExists or outputFile == "" ):
    
            print("This is a listing of files in this path: ")
            flist = os.listdir(outputPath)
            for f in flist:
                print(f)
            yn = input("The file " + outputPath + outputFile +" already exists \n Do you want to overwrite it? ")
            if yn == "y" or yn == "Y":
                fileExists = False
            else:
                outputFile = input("What is the new name for our file? ")
                fileExists = os.path.isfile(outputPath + outputFile)
                

    print("\nWriting " + outputPath+outputFile + "\n\n")


    # form the final scene as a string
    myScene["jsonFileName"] = outputFile
    finalJson = json.dumps(myScene, indent=4)
    f = open(outputPath + outputFile, "w", encoding='utf-8')
    f.write(json.dumps(myScene, ensure_ascii=False, indent=4))
    f.close()

    
def createScene(moduleDescription, sceneData, olist, verbose=False):
    sceneData["objects"] = olist
    
    
    # make a copy of the generic scene for the activity file
    myScene = jdata['genericActivity.json'].copy()

    # apply the module description data ot the sceneObject
    myScene.update(moduleDescription)
    myScene.update(sceneData)


    # we can dump the scene to see what it looks like so far  
    if verbose:
        print("This is the Scene\n\n\n")
        print(json.dumps(myScene, indent=4))    
    
    writeScene(myScene, verbose)    
    
    
# this is a code stub for merging two activity files together
def concatenateScenes( outputPath, outputFile, sceneList):

    print(outputPath, outputFile)
    # open the new file
    f = open(outputPath + outputFile, 'w', encoding='utf-8')
    
    for i in range(len(sceneList)):
        print("  \n scene")
        print(i)
        currentScene = sceneList[i]
        print(currentScene)
        
        # read in the files
        fn1 = outputPath + currentScene
        s1 = jsonFromFile(fn1)

        # Dump the file - but do NOT use any indents.  This will flatten
        # the file into a single line.  We then write it out as a line with a newline character.
        a1 = json.dumps(s1, ensure_ascii=False)
        f.write(a1 + "\n")

    # close it
    f.close()
    
def findKeyDiffs(baseClass, olist):
    # find the differences between two dictionaries
    
    # get the keys for the base class
    baseKeys = list(baseClass.keys())
    
    # find the set of common keys for the objects and the set of unique keys
    commonKeys = []
    uniqueKeys = []
    
    # get the key for each object
    for ii in range(len(olist)):
        cc = olist[i]
        objectKeys = list(cc.keys())
    
        # the union of the keys
        uKeys = list(set(baseKeys + objectKeys))
        
        # the intersection of keys
        iKeys = list(set(baseKeys).intersection(objectKeys))
        
        # find the elements only in one list or the other
        aKeys = list( set(baseKeys)^set(objectKeys))

        commonKeys = commonKeys + uKeys
        uniqueKeys = uniqueKeys + aKeys
        
    
    commonKeys = list(set(commonKeys))
    uniqueKeys = list(set(uniqueKeys))
    
    return commonKeys, uniqueKeys
    
        #for ii in range(len(k1)):
        #    k = k1[ii]
        #    if o1[k] != o2[k]:
        #        print(k)
    
    #


def findDiffs(commonKeys, baseClass, olist):
    
    # for each key
    for c in commonKeys:
        baseValue = baseClass[c]
        
        # are there any differences
        diffs = False
        for i in range(len(olist)):
            newValue = olist[i][c]
            if newValue != baseValue:
                diffs = True


        if diffs:
            print("{0:>15}".format(c), end="")
            for i in range(len(olist)):

                if baseValue != olist[i][c]:

                    print("  X   ", end="")
                else:
                    print("  .   ", end="")

            print()
    print(len(olist))
    
#commonKeys, uniqueKeys = findKeyDiffs(earthObject, sortList)   
#findDiffs(commonKeys, earthObject, sortList)


def createTextLine( textObjectName, theText, fSize, lineNumber, topPosition, rotationAngle, textColor, offSet, parentObject, labelScale):
    
    tdata = {
        "textField": "The Earth and Moon",
        "color": textColor,
        "fontSize": fSize,
        "wrapText": False
        }



    position1 = {"x" :topPosition['x'] + lineNumber * offSet['x'],
                "y": topPosition['y'] + lineNumber * offSet['y'],
                "z":topPosition['z'] + lineNumber * offSet['z']
               }

    tdata["textField"] = theText
    textObject = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": position1.copy(),
        "rotation": rotationAngle,
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": parentObject,
        "name": textObjectName
    }
    
    return textObject.copy()



def createObjectModification(
    name, # object name
    activationCondition, 
    reactivateObject=False, 
    parentObject="[DYNAMIC]", 
    audioClipName = "",
    goCallback ="",
    newColor= None,
    enable=True, 
    newPosition=None, 
    newEulerAngles=None, 
    newScale=None,
    tmp=None):
    
    
    mm = {"name": name,
         "parentObject": parentObject,
         "activationConditions" : activationCondition,
         "reactiveObject": reactivateObject,
          "enable": enable,
          "position": {"x":0.0, "y":0.0, "z":0.0},
          "newPosition": False,
          "eulerAngles": {"x":0.0, "y":0.0, "z":0.0},
          "newEulerAngles": False,
          "scale": {"x":0.0, "y":0.0, "z":0.0},
          "newScale": False,
             }
                        
    if newPosition != None:
        mm["position"] = newPosition
        mm["newPosition"] = True
            
    if newEulerAngles != None:
        mm["eulerAngles"] = newEulerAngles
        mm["newEulerAngles"] = True
            
    if newScale != None:
        mm["scale"] = newScale
        mm["newScale"] = True
            
    if newColor != None:
        mm['color'] = newColor
    
    if tmp != None:
        mm['tmp'] = tmp
        
    return mm.copy()


def createButton(name, 
                 label, 
                 position, 
                 labelOffset = 0.1,
                 scale = 1, 
                 parentObject = "[_DYNAMIC]",
                 color=[255,0,0,255], 
                 eulerAngles= {"x":90, "y":180.0, "z":0.0}):

    
    scriptDataA = {
    "name": "demoButtonActions",
    "callBackObjects": ["demoModule"]
    }
    scriptDataA = json.dumps(scriptDataA) 


    brbData = {
                "type": "Prefabs/demoButtonPrefab",
                "parentName": parentObject,
                "position": position,
                "eulerAngles": eulerAngles,
                "scale": {"x":scale, "y":scale, "z":scale},
                "name": name, 
                "color": color,
                "componentsToAdd": [scriptDataA]
            }
    
    labelPosition  = {"x":0.0, "y":0.0, "z":labelOffset}
    tdata = {
        "textField": "",
        "color": [255,255,0,255],
        "fontSize": 1.0,
        "wrapText": False
        }
    tdata["textField"] = label
    labelScale = 0.6 * scale
    brbLabel = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": labelPosition,
        "eulerAngles": eulerAngles,
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": name,
        "name": name+"Label"  
    }
    return brbData, brbLabel


In [4]:
    
# define the path to access the generic templates and the output files
assetPath = "C:/Users/jfwal/OneDrive/Documents/GitHub/cyberAR/\_Code Device/AR Labs/Assets/Resources/jsonDefinitions/genericDefinitions/"     
outputPath = "C:/Users/jfwal/OneDrive/Documents/GitHub/cyberAR/\_Code Device/AR Labs/Assets/Resources/jsonDefinitions/"  
assetPath = "genericDefinitions/"
outputPath = "./"


# get the list of all the files in the generic directory and load them into a dictionary
flist = listFiles(assetPath,  True)
jdata = loadFileIntoDictionary(assetPath, flist)


# this prints out the assets so you can review them
showFiles = False
if (showFiles):
    for f in flist:
        print("===== file name" + f)
        print("---------\n",f, "\n\n\n", json.dumps(jdata[f], indent=4))
        
#print(json.dumps(jdata["genericActivity.json"]))



genericActivity.json
genericObject.json
genericPointerReceiver.json
genericRigidBody.json
genericScriptSimpleRotation.json


In [5]:
# variables for the objects

orbitalScale = 1.0
timeRate = 1.0
earthSize = 0.08
earthRotationTime = 2.0
moonOrbitalPeriod = 56.0
moonOffSetDistance = 2.4
moonSize = earthSize * 0.25


earthSize = 0.5
moonOffSetDistance = 2.0
moonSize = earthSize * 0.5




############ 


# this uses a simple prefab
sun = {
        "type": "Prefabs/SunPrefab",
        "position": {"x":-39.0, "y":0, "z":0},
        "scale": {"x":5, "y":5, "z":5},
        "name": "Sun",
        "componentsToAdd": []
    }

# make a copy of the generic dictionary for objects    
sunObject = jdata['genericObject.json'].copy()
sunObject.update(sun)



############
scriptData1 = {
    "name": "simpleRotation",
    "timeRate": timeRate,
    "rotationTime": earthRotationTime
}
script1 = json.dumps(scriptData1) 


############
############    
componentsToModify= {
                        "RigidBody": 
                        {
                            "isKinematic": True,
                            "useGravity": False
                        },
                        "PointerReceiver": 
                        {
                        }
                    }




earthPosition = { "x":0.0, 
                 "y": 0.0, 
                 "z":1.5}
earthScale = {"x": earthSize,
            "y": earthSize,
            "z": earthSize }

earth = {
                #"type": "Prefabs/moveableSphere",
                "type": "Prefabs/clickableSphere",
                "position": earthPosition,
                "scale": earthScale,
                "name": "Earth", 
                "parent": "[_DYNAMIC]",
                "texture": "Textures/2k_earth_daymap",
                "componentsToAdd": [script1]
            }


# make a copy of the generic dictionary for objects
#earthObject = jdata['genericObject.json'].copy()

# add the generic component attributes
#earthObject.update(jdata['genericRigidBody.json'])
#earthObject.update(jdata['genericPointerReceiver.json'])
#earthObject.update(earth)

# override the generic component attributes
#for c in componentsToModify.keys():
#    earthObject[c].update(componentsToModify[c])
               
############


moonOffset = {"x":moonOffSetDistance, "y":0.0, "z":0.0}
moonPosition = {"x": earthPosition['x'] + moonOffset['x'],
                "y": earthPosition['y'] + moonOffset['y'],
                "z": earthPosition['z'] + moonOffset['z']
               }


    
moon = {
                #"type": "Prefabs/moveableSphere",
                "type": "Prefabs/clickableSphere",
                "parent": "[_DYNAMIC]",
                "position": moonPosition,
                "scale": {"x":moonSize, "y":moonSize, "z":moonSize},
                "name": "Moon", 
                "texture": "Textures/_k_moon",
                "componentsToAdd": []
            }



# make a copy of the generic dictionary for objects    
#moonObject = jdata['genericObject.json'].copy()

# add the generic component attributes
#moonObject.update(jdata['genericRigidBody.json'])
#moonObject.update(jdata['genericPointerReceiver.json'])
#moonObject.update(moon)

# override the generic component attributes
#for c in componentsToModify.keys():
#    moonObject[c].update(componentsToModify[c])




############ 


# make a copy of the generic dictionary for objects    
moonObject = jdata['genericObject.json'].copy()

# add the generic component attributes
moonObject.update(jdata['genericRigidBody.json'])
moonObject.update(jdata['genericPointerReceiver.json'])
moonObject.update(moon)

# override the generic component attributes
for c in componentsToModify.keys():
    moonObject[c].update(componentsToModify[c])


############     
   
tdata = {
        "textField": "8000 miles",
        "color": [255,255,0,255],
        "fontSize": 1.0,
        "wrapText": False
        }


moonLabelPosition = {"x":0.0, "y":1, "z":0.0}

tdata["textField"] = "14 days after \n New Moon"
tdata["fontSize"] = 1
tdata["color"] = [0, 255, 255, 0]
labelScale =2
moonLabelTop = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": moonLabelPosition,
        "eulerAngles": {"x": 0.0, "y": 90.0, "z": 0.0},
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": "Moon",
        "name": "moonLabelTop"     
}


############ 


moonLabelPosition = {"x":0.0, "y":-0.75, "z":0.0}

tdata["textField"] = "Full Moon"
labelScale = 1.5
moonLabelBottom = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": moonLabelPosition,
        "eulerAngles": {"x": 0.0, "y": 90.0, "z": 0.0},
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": "Moon",
        "name": "moonLabelBottom"     
    }


############ 



robotSize = moonSize / 2.0
robotPosition = {"x":moonSize, "y":0.0, "z":0.0}

robotSize = moonSize * 1.5
robotPosition = {"x":-moonSize*2, "y":0.0, "z":0.0}



robot = {
                "type": "Prefabs/robotIdle",
                "parentName": "Moon",
                "position": robotPosition,
                "eulerAngles": {"x":-90.0, "y":0.0, "z":90.0},
                "scale": {"x":robotSize, "y":robotSize, "z":robotSize},
                "name": "Robot", 
                "componentsToAdd": []
            }

############     

textObjectName = "sectionLabel"
theText = "The View from the Moon"
fSize=  1.0
lineNumber = 0
topPosition = {"x":0.0, "y": 0.75, "z":2.50}
rotationAngle = {"x": 0.0, "y": 0.0, "z": 0.0}
textColor = [255,255,255,255]
offSet = {"x":0.0, "y":-0.15, "z":0.0}
parentObject = "[_DYNAMIC]"
labelScale = 0.6



sectionLabel = createTextLine( textObjectName, 
               theText, 
               fSize, 
               lineNumber, 
               topPosition, 
               rotationAngle, 
               textColor, 
               offSet, 
               parentObject, 
               labelScale)

####

textObjectName = "taskLabel"
theText = "Introduction"
fSize=  1
lineNumber = 1
labelScale = 0.6



taskLabel = createTextLine( textObjectName, 
               theText, 
               fSize, 
               lineNumber, 
               topPosition, 
               rotationAngle, 
               textColor, 
               offSet, 
               parentObject, 
               labelScale)
 

####

#iboxPosition = {"x":-0.1, "y": 0.3, "z":1.5}
#ibox = {
#                "type": "Prefabs/instructionCanvas",
#                "parentName": "[_DYNAMIC]",
#                "position": iboxPosition,
#                "name": "instructionCanvas", 
#                "componentsToAdd": []
#            }

In [6]:


#olist = [sun, earth, moon, robot, moonLabelTop, moonLabelBottom, taskLabel, sectionLabel] #, sun, earthSizeLabel, moonLabel, moonSizeLabel]
olist = [sun, earth, moon, moonLabelTop, moonLabelBottom, taskLabel, sectionLabel] #, sun, earthSizeLabel, moonLabel, moonSizeLabel]

moduleButtonOffset = 0.8

name = "brbNextModule"
label = "Next\n Module"
labelOffset = 0.13
scale= 1.0

parentObject = "sectionLabel"
position = {"x": moduleButtonOffset, "y": 0, "z":0}
o1, o2 =  createButton(name, label, position, labelOffset, scale, parentObject=parentObject)
o1["childColor"] = [0, 255, 255, 255]
o1["childName"] = "button"
olist.append(o1)
olist.append(o2)


name = "brbPreviousModule"
label = "Previous \n Module"
labelOffset = 0.13
scale= 1.0
parentObject = "sectionLabel"
position = {"x": -moduleButtonOffset, "y": 0, "z":0}
o1, o2 =  createButton(name, label, position, labelOffset, scale=scale, parentObject=parentObject)
o1["childColor"] = [0, 255, 255, 255]
o1["childName"] = "button"
olist.append(o1)
olist.append(o2)

##
clipButtonOffset = 0.5
parentObject = "taskLabel"
labelOffset = 0.13


name = "brbNextClip"
label = "Next"
position = {"x": clipButtonOffset, "y": 0, "z":0}
o1, o2 =  createButton(name, label, position, parentObject=parentObject)
o1["childColor"] = [0, 255, 0, 255]
o1["childName"] = "button"
olist.append(o1)
olist.append(o2)


name = "brbPreviousClip"
label = "Previous"
position = {"x": -clipButtonOffset, "y": 0, "z":0}
o1, o2 = createButton(name, label, position, parentObject=parentObject)
o1["childColor"] = [0, 255, 0, 255]
o1["childName"] = "button"
olist.append(o1)
olist.append(o2)

####

instructions = "In the lab handout, go to the section on Synchronous \n rotation.  Go through all the Moon Phases using \nthe Next and Previous buttons.  In the lab handout, \ndraw a stick figure showing the direction \nit is facing on at each phase. "




#olist.append(instructions)

In [7]:
clipList = []
               # 1   2   3   4   5   6   7   8   9  10  11 
#timesList2 = [0, 14, -1, -1, 12, 11,  26, -1, 10, 14, 33, -1]
#timesList = [0, 14, 10, 9, 12, 11, 26, 10, 14, 33, 9, 19]

clipMax = 9

     #                X   X                               X
     # animate                           X            

        # now that you have seen this demo, it's time to do some work.
        # answer the questions on the handout about the sizes and scales in the Earth Moon system.
        # you can go to previous parts of the demonstration if you need to review the data.


textColor = [255,255,255,255]
fSize=  1
tdata = {
        "textField": "The Earth and Moon",
        "color": textColor,
        "fontSize": fSize,
        "wrapText": False
        }

        
clipList = []

oc0 = createObjectModification("[_DYNAMIC]", 0,  newPosition={"x":0.0, "y":1.5, "z":0.0})
tdata['fontSize'] = 1
tdata['textField'] = "14 days after \n New Moon"
tdata["parentObject"] = None

oc0a = createObjectModification(
        "moonLabelTop", 
        0,  
        tmp =json.dumps(tdata)
        )
tdata['textField'] = "Full Moon"
tdata['fontSize'] = 1
oc0b = createObjectModification(
        "moonLabelBottom", 
        0,  
        tmp =json.dumps(tdata)
        )


#clipObj = {"clipName":"clip0", 
#    "audioClipString":"Audio/moonLab/moonLab_1_0_0",
#    "timeToEnd":14, 
#    "autoAdvance": True, 
#    "goNext": "brbNext",
#    "goPrevious":"brbPrevious",
#    "objectChanges": [oc0, oc0a, oc0b] }
        

topTitle = [
    "14 days after \n New Moon",
    "17.5 days after \n New Moon",
    "21 days after \n New Moon",
    "24.5 days after \n New Moon",
    "New Moon",
    "3.5 days after \n New Moon",
    "7 days after \n New Moon",
    "11.5 days after \n New Moon",

    ]

bottomTitle = [
    "Full Moon",
    "Waning Gibbous",
    "Third Quarter",
    "Waning Crescent",
    "New Moon",
    "Waxing Crescent",
    "First Quarter",
    "Waxing Gibbous"
    ]


###
cmod = createObjectModification(
        "MainInstructions", 
        0
        )

#cmod['canvasText'] = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum."
#instructions = "In the lab handout, go to the section on Synchronous \n rotation. Explore all the Moon Phases using \nthe Next and Previous buttons.  In the lab handout, \ndraw a stick figure showing the direction \nit is facing on at each phase. "



verbalInstructions = """
Let's change perspective and see what the Earth looks like
as the Moon goes through its monthly cycle.  
In the last module, we looked at Moon Phases.  We will
now look the Phases the Earth as viewed from the Moon.

When you are viewing the moon phases, you needed to stand in the location
of the Earth. To view the phases of Earth, you will need to position yourself
in the location of the moon.
 

In the lab handout, go to the section on the View from the Moon. 
As before, you can use the Next and Previous buttons floating above
the Earth to move the moon through it's orbit.  At each of these locations,
note how the Earth appears as viewed from the Moon. Again - make sure you are 
standing on the location of the Moon to do these drawings.
Draw the shadow on the diagram to show how the phases change during 
the month.  



3_1_0
When you completed exploring all the moon phases, a red Next Module
will appear.  Click on this button to go to the next module in this lab.


"""




instructions = """In the lab handout, go to the section on the
'The View from the Moon.'  As before, explore all the Moon Phases 
using the Next and Previous buttons.  

In the lab handout, sketch how the EARTH appears as it is viewed from Moon.
You are actually diagraming "Earth Phases" instead of Moon Phases for this
part of the lab.

When you finish looking at all the Earth phases, a red Next Module
will appear.  Click on this button to go to the next module in this lab."""

cmod['canvasText'] = instructions

timeToEnd = 53    
clipObj = {"clipName":"clip0", 
    "timeToEnd": timeToEnd, 
    "autoAdvance": True, 
    "objectChanges":[cmod]
          }



clipList.append(clipObj)



r = 2.0
autoAdvance = False
#autoAdvance = True
timeToEnd = -1
timeToEnd = 3
for i in range(0, 8): 

    theta = np.pi * 2.0 * float(i) / 8.0
    thetaRot = -360.0 * float(i) / 8.0
    xpos = r * np.cos(theta) + earthPosition["x"]
    zpos = r * np.sin(theta) + earthPosition["z"]
    ypos = 0.0 + earthPosition["y"]
    
    oc1 = createObjectModification(
        "Moon", 
        0,  
        newPosition = {"x": xpos, "y":ypos, "z":zpos},
        newEulerAngles = {"x": 0.0, "y": thetaRot, "z": 0.0}
    )

    print(i, topTitle[i])
    tdata['fontSize'] = 1
    tdata["textField"] = topTitle[i]
    oc2 = createObjectModification(
        "moonLabelTop", 
        0,  
        tmp =json.dumps(tdata)

    )
    tdata['textField'] = bottomTitle[i]
    tdata['fontSize'] = 1
    oc3 = createObjectModification(
        "moonLabelBottom", 
        0,  
        tmp =json.dumps(tdata)
        )


#    if i == 0:    
#        tdata['fontSize'] = 1
#        tdata['textField'] = bottomTitle[0]
#        tdata["parentObject"] =  "[_DYNAMIC]"
#    else:
#        tdata['fontSize'] = 1
#        tdata['textField'] = bottomTitle[i-1]
#        tdata["parentObject"] =  "[_DYNAMIC]"
    
    tdata['fontSize'] = 1
    tdata['textField'] = bottomTitle[i]
    tdata["parentObject"] =  "[_DYNAMIC]"

    ##tdata['textField'] = "ducky " + str(i)
    oc0a = createObjectModification(
        "taskLabel", 
        0,  
        tmp =json.dumps(tdata)
        )    
    
    clipObj = {"clipName":"clip" + str(i+1), 
         "timeToEnd": timeToEnd, 
         "autoAdvance": autoAdvance, 
         "objectChanges": [oc1, oc2, oc3, oc0a.copy()]  }
    
    clipList.append(clipObj)

############ button and navigation management

for i in range(len(clipList)):
    

    oc0c = createObjectModification(
        "brbPreviousClip", 
        0 
        )
    

    ###
    oc0d = createObjectModification(
        "brbNextClip", 
        0 
        )    
  
    ###
    oc0e = createObjectModification(
        "brbNextModule", 
        0 
        )    

    # previous clip
    if i == 0:
        oc0c["scale"] = {"x": 0.01, "y":0.01, "z":0.01}
        oc0c["newScale"] = True     
    else:
        oc0c["scale"] = {"x": 1, "y":1, "z":1}
        oc0c["newScale"] = True     

    # next clip
    if i != len(clipList) -1:
        oc0d["scale"] = {"x": 1, "y":1, "z":1}
        oc0d["newScale"] = True
    else:
        oc0d["scale"] = {"x": 0.01, "y":0.01, "z":0.01}
        oc0d["newScale"] = True       

    # next module
    #oc0e["childColor"] = [255, 0, 0, 255]
    #oc0e["childName"] = "button"
    if i == len(clipList) -1:
        oc0e["childColor"] = [255, 0, 0, 255]
        oc0e["childName"] = "button"
        oc0e["scale"] = {"x": 1, "y":1, "z":1}
        oc0e["newScale"] = True  
    else:
        oc0e["scale"] = {"x": 0.01, "y":0.01, "z":0.01}
        oc0e["newScale"] = True  
    
    
    clipList[i]["objectChanges"].append(oc0c)
    clipList[i]["objectChanges"].append(oc0d)
    clipList[i]["objectChanges"].append(oc0e)

clipList[-1]["audioClipString"] = "Audio/moonLab/moonLab_3_1_0"
clipList[0]["audioClipString"] = "Audio/moonLab/moonLab_5_0_0"


for c in clipList:
    print( json.dumps(c, indent=4))
    
    

    

0 14 days after 
 New Moon
1 17.5 days after 
 New Moon
2 21 days after 
 New Moon
3 24.5 days after 
 New Moon
4 New Moon
5 3.5 days after 
 New Moon
6 7 days after 
 New Moon
7 11.5 days after 
 New Moon
{
    "clipName": "clip0",
    "timeToEnd": 53,
    "autoAdvance": true,
    "objectChanges": [
        {
            "name": "MainInstructions",
            "parentObject": "[DYNAMIC]",
            "activationConditions": 0,
            "reactiveObject": false,
            "enable": true,
            "position": {
                "x": 0.0,
                "y": 0.0,
                "z": 0.0
            },
            "newPosition": false,
            "eulerAngles": {
                "x": 0.0,
                "y": 0.0,
                "z": 0.0
            },
            "newEulerAngles": false,
            "scale": {
                "x": 0.0,
                "y": 0.0,
                "z": 0.0
            },
            "newScale": false,
            "canvasText": "In the lab handout, 

In [8]:
bottomTitle

['Full Moon',
 'Waning Gibbous',
 'Third Quarter',
 'Waning Crescent',
 'New Moon',
 'Waxing Crescent',
 'First Quarter',
 'Waxing Gibbous']

In [9]:
# The moduleDescription is the text information that is NOT used by the application at runtime.
# It is only data that we will reference in the database of modules and editing.

moduleDescription = {
        "jsonFileName": "phasesOfEarth.json",
        "moduleName": "The View from the Moon",
        "description": "Earth Phases See From the Moon",  
        "author": "John Wallin",
        "authorInstitution": "Middle Tennessee State University",
        "dateCreated": time.strftime("%c")
    }


# MediaInfo[ { "id:", "type": 1}
# MediaURL

# sceneData contains information about prefabs and prefab specific data
sceneData = {
        "prefabName": "demoPrefab",
        "specificName": "Earth Phases See From the Moon",  # this will appear on the navigation window
        "educationalObjectives": ["Students will observe the phases of the Earth as viewed from the Moon"],
        "instructions": [instructions],
    
        # specific to the demo prefab
        "introAudio": "",
        "createObjects": True,
        "destroyObjects": True,
        "restoreLights": True,
        "timeToEnd": 2000,
        "objects": [],
        "useSunlight": True,
        "clips":clipList
}



createScene(moduleDescription, sceneData, olist, verbose=False)

True
./phasesOfEarth.json

Writing ./phasesOfEarth.json




In [10]:

outputFile = "demo10.json"
#concatenateScenes(outputPath, outputFile, ['basketBallDemo.json', 'miniEarthMoon.json', 'earthMoon.json'])

outputPath = ""
#concatenateScenes(outputPath, outputFile, ['moonEarthIntroduction.json','moonEarthSizes.json','lunarRotation.json'] )
concatenateScenes(outputPath, outputFile, ['phasesOfEarth.json'] )

#concatenateScenes(outputPath, outputFile, ['moonEarthIntroduction.json'] )

                  #,'miniEarthMoon.json']  
                  # 'earthMoon.json'])


 demo10.json
  
 scene
0
phasesOfEarth.json


In [11]:

'''
zStart = 3.0


topPosition1 = {"x": 0.0 ,
                "y": +0.5,
                "z": zStart
               }
rotationAngle = {"x": 0.0,
                "y": 0.0,
                "z": 0.0
               }
textColor = [255, 255, 0, 255]
labelScale =2.0
fSize = 1.0
offSet = {"x": 0.0 ,
            "y": -0.2,
            "z": 0.0
            }
parentObject = "[_DYNAMIC]"

textObjectName = "title"
theText = "The Earth and Moon"
lineNumber = 0

textObjectList = []
to0 = createTextLine( textObjectName, theText, fSize, lineNumber, topPosition1, rotationAngle, textColor, offSet, parentObject, labelScale)
textObjectList.append(to0)



topPosition = {"x": 0.0 ,
                "y": +0.4,
                "z": zStart
              }
fSize = 0.65
textObjectName = "line1"
theText = "Learn about Sizes and Scales"
lineNumber = 1
to1 = createTextLine( textObjectName, theText, fSize, lineNumber, topPosition, rotationAngle, textColor, offSet, parentObject, labelScale)
textObjectList.append(to1)

fSize = 0.65
textObjectName = "line2"
theText = "Determine how the Moon rotates"
lineNumber = 2
to2 = createTextLine( textObjectName, theText, fSize, lineNumber, topPosition, rotationAngle, textColor, offSet, parentObject, labelScale)
textObjectList.append(to2)


fSize = 0.65
textObjectName = "line3"
theText = "Diagram how sunlight illuminates the moon "
lineNumber = 3
to3 = createTextLine( textObjectName, theText, fSize, lineNumber, topPosition, rotationAngle, textColor, offSet, parentObject, labelScale)
textObjectList.append(to3)

fSize = 0.65
textObjectName = "line4"
theText = "Learn the names of moon phases and their order"
lineNumber = 4
to4 = createTextLine( textObjectName, theText, fSize, lineNumber, topPosition, rotationAngle, textColor, offSet, parentObject, labelScale)
textObjectList.append(to4)

fSize = 0.65
textObjectName = "line5"
theText = "Measure orbital periods"
lineNumber = 5
to5 = createTextLine( textObjectName, theText, fSize, lineNumber, topPosition, rotationAngle, textColor, offSet, parentObject, labelScale)
textObjectList.append(to2)


textPosition = { "x":1.5, 
                 "y": 0.0, 
                 "z":zStart}
makerSize = 0.1
markerBall = {
                "type": "Prefabs/tinySphere",
                "parent": parentObject,
                "position": topPosition,
                "scale": {"x":makerSize, "y":makerSize, "z":makerSize},
                "name": "markerBall", 
                "componentsToAdd": []    
    
    }



yvalue = [0.2, 0.0, -0.2, -0.4, -0.6]
for i in range(len(yvalue)):
    textPosition = { "x":-1.5, 
                 "y": yvalue[i], 
                 "z":zStart}
    markerBall = {
                "type": "Prefabs/tinySphere",
                "parent": parentObject,
                "position": textPosition,
                "scale": {"x":makerSize, "y":makerSize, "z":makerSize},
                "name": "markerBall"+str(i+1), 
                "color": [255, 0, 0, 255],
                "componentsToAdd": []    
    
    }    
    textObjectList.append(markerBall)
    
    
olist = olist #+ textObjectList
#for ow in olist:
#    print(json.dumps(ow, indent=4))
'''
print("DONE!")

DONE!


In [12]:
moonSize


0.25